In [1]:
import lightgbm
import pandas as pd

In [7]:
init_record = pd.read_csv("../../Data/LWB_Monthly_data.csv")


In [8]:
init_record['_key_occurreddate_month']=init_record['_key_occurreddate_month'].astype("datetime64[ns]")
#init_record = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])

In [4]:
init_record.shape

(123311, 315)

## Use tsfresh to extract features

In [34]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh.utilities.distribution import MultiprocessingDistributor
import re
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_curve
from sklearn import metrics
import matplotlib.pyplot as plt

In [14]:
init_record.columns = [re.sub(r"__*", "_", name) for name in init_record.columns]

In [15]:
Distributor = MultiprocessingDistributor(n_workers=4,
                                         disable_progressbar=False,
                                         progressbar_title="Feature Extraction")
settings = MinimalFCParameters()

In [16]:
train_extracted = extract_features(init_record, column_id="_key_client_id", column_sort="_key_occurreddate_month",
                                   default_fc_parameters=settings, distributor=Distributor)

Feature Extraction: 100%|██████████| 20/20 [56:14<00:00, 168.73s/it]


In [17]:
from sklearn.ensemble import IsolationForest

In [18]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(train_extracted)

IsolationForest(contamination=0.1, n_estimators=50)

In [22]:
y_hat = model.predict(train_extracted)

In [27]:
data = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])
y = list(data.groupby(level=0).apply(lambda group: group.iloc[-1])['response_variable'].astype('int32'))

In [29]:
len(y)

13405

In [24]:
len(y_hat)

13405

In [32]:
y_hat = np.where(y_hat == -1, 1, 0)  

In [35]:
f1_score(y, y_hat)

0.251340083382966

In [36]:
confusion_matrix(y,y_hat)

array([[11937,  1129],
       [  128,   211]])